In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 1
chi = 0.3
positions = np.array(natsorted(([str(element) for element in np.around(np.linspace(-40,40,16),decimals=2)])))
positions

In [ ]:
data = []

for s,path_direction in enumerate([
                        r"H:\phd stuff\tidy3d\data\08_27_2024 chi 0.3 N1000 Moving Beam Experiments\z_incidence",
                        
                       ]):

      folder_path = f"{path_direction}"
      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                sim_data = structure_1.sim_data
                sim_data0 = structure_1.sim_data0
                field = sim_data.at_centers("freq_monitorFieldOut")
                field0 =sim_data0.at_centers("freq_monitorFieldOut")
                required_components = ["E" + c for c in "xyz"]
                field_components = ([field[c] for c in required_components])
                field_components0 = ([field0[c] for c in required_components])
                data += [
                      {          
                                "frequencies":a/(td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)),
                                "focal_point":positions[i],
                                "sim":sim_data,
                                "Re_E_input":sum(f.real for f in field_components0).squeeze(),
                                "Im_E_input":sum(f.imag for f in field_components0).squeeze(), 
                                "Re_E_total":sum(f.real for f in field_components).squeeze(),
                                "Im_E_total":sum(f.imag for f in field_components).squeeze()
                      }
                ]


In [24]:
freqs = a/(td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs))
freqs_raw = (td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs))
freqs[0]

0.28500000000000003

In [ ]:
coords = sim_data["freq_monitorFieldOut"].Ex
min_x,max_x,min_y,max_y = np.min(coords["x"]),np.max(coords["x"]), np.min(coords["y"]),np.max(coords["y"])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as mpath

In [ ]:
def plot_fields(index_freq:int=-1,index_pos:int = -1):
    index=index_pos

    fig1, ax_1 = plt.subplots()
    ax1=sim_data.simulation.plot_structures_eps(freq=freqs_raw[index_freq], cbar=False, z=0, ax=ax_1,
                reverse=False)

    # Create a new figure for plotting the shapes
    patches_list = ax1.patches

    



    fig, ax = plt.subplots(ncols=6,nrows=1,figsize=(24,10))
    extent=(min_x,max_x,min_y,max_y)
    fig.suptitle(fr'$\chi=$'+str(chi)+fr'; $y_{0}=${data[index]["focal_point"]}'+fr'; $\nu=${freqs[index]:.3g}', fontsize=16)
    plt.xticks(rotation=20)

    #input field Re(E)
    absmax=np.amax(np.abs(data[index]["Re_E_input"][:,:,index_freq]))
    absmin=np.amin((data[index]["Re_E_input"][:,:,index_freq]))


    ax1=(data[index]["Re_E_input"]).T.isel(f=index_freq).plot.pcolormesh(ax=ax[0],cmap="RdBu",vmin=absmin,vmax=absmax)


    ax[0].set_title(r'Re $\left(E_{0}\right)$')

    #Total field Re(E)
    absmax=np.amax(np.abs(data[index]["Re_E_total"][:,:,index_freq]))
    ax2=(data[index]["Re_E_total"]).T.isel(f=index_freq).plot.pcolormesh(ax=ax[1],cmap="RdBu",vmin=-1*absmax,vmax=absmax)
    ax[1].set_title(r'Re $\left(E\right)$')



     #input field Im(E)
    absmax=np.amax(np.abs(data[index]["Im_E_input"][:,:,index_freq]))
    ax3=(data[index]["Im_E_input"]).T.isel(f=index_freq).plot.pcolormesh(ax=ax[2],cmap="RdBu",vmin=-1*absmax,vmax=absmax)
    ax[2].set_title(r'Im $\left(E_{0}\right)$')


    #Total  field Im(E)
    absmax=np.amax(np.abs(data[index]["Im_E_total"][:,:,index_freq]))
    ax4=(data[index]["Im_E_total"]).T.isel(f=index_freq).plot.pcolormesh(ax=ax[3],cmap="RdBu",vmin=-1*absmax,vmax=absmax)

    ax[3].set_title(r'Im $\left(E\right)$')

    #square of the input field     
    map_to_plot=np.sqrt(data[index]["Re_E_input"][:,:,index_freq]**2+data[index]["Im_E_input"][:,:,index_freq]**2)
    absmax=np.amax(map_to_plot)
    ax5=(data[index]["Re_E_input"]).T.isel(f=index_freq).plot.pcolormesh(ax=ax[4],cmap="magma",vmin=0,vmax=absmax)

    ax[4].set_title(r'$\left|E_{0}\right|$')                  

    #square of the total field    
    map_to_plot=np.sqrt(data[index]["Re_E_total"][:,:,index_freq]**2+data[index]["Im_E_total"][:,:,index_freq]**2)
    absmax=np.amax(map_to_plot)
    ax6=(data[index]["Re_E_total"]).T.isel(f=index_freq).plot.pcolormesh(ax=ax[5],cmap="magma",vmin=0,vmax=absmax)

    ax[5].set_title(r'$\left|E\right|$')                       


    #tilting lavels axes x
    for i in (0,1,2,3,4,5):
        #for j in (0,1):
        ax[i].tick_params(axis='x', rotation=70)
        ax[i].set_xlabel("X")
        ax[i].set_ylabel("Y")
        for patch in patches_list:
            path = patch.get_path()  
            # Reconstruct the shape
            if i >3:
                new_patch = patches.PathPatch(path, edgecolor= (1, 1, 1, 0.1), facecolor='none')
            else:
                new_patch = patches.PathPatch(path, edgecolor= (0, 0, 0, 0.1), facecolor='none')


            ax[i].add_patch(new_patch)



    fig.tight_layout()

    return 

In [ ]:
import numpy as np    
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

interact_manual(plot_fields,
         index_pos=(0,positions.shape[0],1),
         index_freq=(0,freqs_raw.shape[0],1))

In [27]:
field_data = {
    "Ez":data[12]["sim"]["freq_monitorFieldOut"].Ez.values[:,:,:,0].squeeze(),
    "x":data[12]["sim"]["freq_monitorFieldOut"].Ez.x,
    "y":data[12]["sim"]["freq_monitorFieldOut"].Ez.y,
}

In [28]:
create_hdf5_from_dict(field_data,"field_data_pos_-24_freq_nu_0.285.h5")